In [51]:
import os, sys
import numpy as np
import pandas as pd
import datetime 
import plotly.express as px
import matplotlib.pyplot as plt
import yfinance as yf

%matplotlib qt

In [2]:
#l_assets = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']
l_assets = ['AAPL', 'JPM', 'WMT', 'TGT', 'MSFT', 'AMGN']

dt_end = datetime.date.today()
dt_start = dt_end - datetime.timedelta(days=500)

dt_start = datetime.date(2000,1,3)
dt_end = datetime.date(2018,8,24)

df = yf.download(l_assets, dt_start, dt_end)
df_ts = df['Adj Close']

[*********************100%***********************]  6 of 6 completed


In [3]:
df_rtn = np.log(df_ts).diff(1).dropna() # take the log returns
df_rtn.head()

,AAPL,AMGN,JPM,MSFT,TGT,WMT
Date,,,,,,
2000-01-04,-0.088078,-0.079546,-0.022186,-0.034364,-0.044333,-0.038136
2000-01-05,0.014527,0.033830,-0.006192,0.010489,-0.022926,-0.020619
2000-01-06,-0.090514,0.016495,0.014098,-0.034072,-0.048466,0.010854
2000-01-07,0.046282,0.106587,0.018206,0.012984,0.050319,0.072845
2000-01-10,-0.017745,0.058893,-0.017331,0.007264,-0.019636,-0.018417


In [48]:
daily_returns = pd.read_csv('daily_returns.csv', index_col=0)
daily_returns.index = pd.to_datetime(daily_returns.index)

In [52]:
df_c = pd.concat([daily_returns['AAPL'], df_rtn['AAPL']], axis=1)
plt.figure()
plt.plot(daily_returns.index, daily_returns['AAPL'])
plt.plot(df_rtn.index, df_rtn['AAPL'])
plt.show()

In [59]:
df_c = pd.concat([daily_returns['AAPL'], df_rtn['AAPL']], axis=1).dropna()

In [64]:
plt.figure()
plt.plot(df_c.iloc[:,0], df_c.iloc[:,0], '.')
plt.plot(df_c.iloc[:,0], df_c.iloc[:,1], '.')

plt.show()

In [66]:
df_c.mean(), df_c.median()

(AAPL    0.001201
 AAPL    0.000876
 dtype: float64,
 AAPL    0.000756
 AAPL    0.000770
 dtype: float64)

In [61]:
daily_returns.shape

(4692, 6)

In [55]:
df_rtn['AAPL'].mean() * 250

0.21897272810668886

In [38]:
#-- Get annualised mean returns
mus = (1+daily_returns.mean())**252 - 1

#-- Get covariances
#- Multiply by 252 to annualise it (square root time for volatility but no square root for variance)
#- Note: 252 trading days in a year
#- https://quant.stackexchange.com/questions/4753/annualized-covariance
cov = daily_returns.cov()*252

In [42]:
ds_mu, mus, daily_returns.mean() * 252

(AAPL    0.218973
 AMGN    0.069376
 JPM     0.074803
 MSFT    0.055162
 TGT     0.064395
 WMT     0.036949
 dtype: float64,
 AAPL    0.359555
 JPM     0.127543
 WMT     0.046851
 TGT     0.103386
 MSFT    0.083253
 AMGN    0.122341
 dtype: float64,
 AAPL    0.307345
 JPM     0.120069
 WMT     0.045791
 TGT     0.098403
 MSFT    0.079981
 AMGN    0.115443
 dtype: float64)

In [29]:
df_cov

,AAPL,AMGN,JPM,MSFT,TGT,WMT
AAPL,0.180858,0.036301,0.055467,0.052295,0.035084,0.023148
AMGN,0.036301,0.102399,0.039975,0.035480,0.027477,0.021868
JPM,0.055467,0.039975,0.152977,0.053141,0.054934,0.032287
MSFT,0.052295,0.035480,0.053141,0.092351,0.033263,0.023556
TGT,0.035084,0.027477,0.054934,0.033263,0.102976,0.043545
WMT,0.023148,0.021868,0.032287,0.023556,0.043545,0.056861


In [15]:
# calculate mean and cov
DAYS_IN_ANNUM = 250
ds_mu = df_rtn.mean() * DAYS_IN_ANNUM
df_cov = df_rtn.cov() * DAYS_IN_ANNUM

# Portfolios
Fully invested portfolios
* generate non-negative weights
* sum to one

In [23]:
np.random.seed(1234) # fix the seed (for re-producible)
num_portfolios = 1000

l_kw_ports = [] # list of portfolios and their characteristics
num_assets = len(l_assets)
for p in range(num_portfolios):
    v_w = np.random.rand(num_assets)
    v_w /= np.sum(v_w)

    mu_p = np.dot(ds_mu.values, v_w)
    var_p = np.dot(v_w, np.dot(df_cov.values, v_w))

    l_kw_ports.append({'w':v_w, 'mu': mu_p, 'var': var_p})




In [26]:
for p in range(1):
    v_w = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    v_w /= np.sum(v_w)

    mu_p = np.dot(ds_mu.values, v_w)
    var_p = np.dot(v_w, np.dot(df_cov.values, v_w))

    l_kw_ports.append({'w':v_w, 'mu': mu_p, 'var': var_p})

In [27]:
v_mu_p = np.array([kw['mu'] for kw in l_kw_ports])
v_std_p = np.sqrt(np.array([kw['var'] for kw in l_kw_ports]))
v_str_w = [str(kw['w']) for kw in l_kw_ports]


In [28]:
rf_rate = 0.01
v_sharp = (v_mu_p - rf_rate) / v_std_p

df_px = pd.DataFrame({'mu':v_mu_p, 'std':v_std_p, 'sharp ratio':v_sharp, 'weight':v_str_w})
fig = px.scatter(df_px, x='std', y='mu', color='sharp ratio')
fig.update_yaxes(range=[0.0, 0.5])
fig.update_xaxes(range=[0.0, 0.5])
fig.show()


In [54]:
var_p

0.06568752920506735

In [52]:
np.dot(v_w, np.dot(cov.values, v_w))

0.06568751740128774

In [43]:
np.diff(v_w).sum()

1.0